In [ ]:
import ray

In [ ]:
ray.init("ray://localhost:10001")

(ServeController pid=293) INFO 2025-07-10 02:53:07,413 controller 293 -- Draining proxy on node 'fb8180e7e373de244188a0116034811028ec39aa8f3fccf3b6e853bb'.
(ServeController pid=293) INFO 2025-07-10 02:53:07,414 controller 293 -- Removing proxy on removed node 'fb8180e7e373de244188a0116034811028ec39aa8f3fccf3b6e853bb'.
Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2025-07-10 15:23:10,238	WARNING dataclient.py:403 -- Encountered connection issues in the data channel. Attempting to reconnect.


In [ ]:
from ray import serve
from ray.serve.llm import LLMConfig, build_openai_app, LLMServer, LLMRouter

In [ ]:
lm_config = LLMConfig(
    model_loading_config=dict(
        model_id="mistral-7b2",
        model_source="/models/llms/mistral-7b/e0bc86c23ce5aae1db576c8cca6f06f1f73af2db",
    ),
    deployment_config=dict(
        autoscaling_config=dict(
            min_replicas=1, max_replicas=1,
        )
    ),
    # Pass the desired accelerator type (e.g. A10G, L4, etc.)
    accelerator_type="T4",
    # You can customize the engine arguments (e.g. vLLM engine kwargs)
    engine_kwargs=dict(
        tensor_parallel_size=1,
    ),
)

In [ ]:
deployment = LLMServer.as_deployment(lm_config.get_serve_options(name_prefix="vLLM2:")).bind(lm_config)
llm_app = LLMRouter.as_deployment().bind([deployment])
serve.run(llm_app, blocking=True)

In [6]:
! pip install openai

I0000 00:00:1752147240.522755   57371 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [11]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="my-mistral-model",
    messages=[
        {"role": "user", "content": "Tell me a short story about king and queen"}
    ]
)

print(completion.choices[0].message)

{"asctime": "2025-07-10 17:20:50,940", "levelname": "INFO", "message": "HTTP Request: POST http://localhost:8000/v1/chat/completions \"HTTP/1.1 200 OK\"", "filename": "_client.py", "lineno": 1025, "timestamp_ns": 1752148250940208724}


ChatCompletionMessage(content=" Title: The Gift of Gratitude\n\nOnce upon a time, in the vast and prosperous kingdom of Serendell, King Eldrone and Queen Evangeline ruled with wisdom and kindness. The kingdom was rich in resources, and its people lived happily, cherishing their beloved monarchs.\n\nKing Eldrone was a man of valor, and Queen Evangeline was a woman of grace and compassion. They were admired and loved by all, and their generous hearts were known far and wide.\n\nOne day, a humble traveler arrived at the royal court. He was a weaver from a distant village, seeking the king and queen's help. He told them about a terrible drought that had struck his home, and the villagers were suffering.\n\nThe king and queen, moved by the traveler's plea, decided to help. King Eldrone dispatched soldiers to the drought-stricken village, equipped with tools and supplies to dig a well.\n\nWeeks later, the well was complete, and the village was able to water its crops once again. The people r